# ACRO Tests

In [1]:
import os
import sys
import pandas as pd
import numpy as np

In [2]:
sys.path.insert(0, os.path.abspath(".."))

In [3]:
from acro import ACRO, add_constant, utils

### Instantiate ACRO

In [4]:
acro = ACRO(suppress=False)

INFO:acro:version: 0.4.2
INFO:acro:config: {'safe_threshold': 10, 'safe_dof_threshold': 10, 'safe_nk_n': 2, 'safe_nk_k': 0.9, 'safe_pratio_p': 0.1, 'check_missing_values': False}
INFO:acro:automatic suppression: False


### Load test data

In [5]:
path = os.path.join("../data", "test_data.dta")
df = pd.read_stata(path)
df.head()

,charity,grant_type,index,year,inc_activity,inc_grants,inc_donations,inc_other,inc_total,total_costs,...,sh_staff_grants_given,sh_assets_grants_given,sh_income_balance,sh_staff_balance,sh_assets_balance,sh_income_assets,sh_staff_assets,sh_income_staff_costs,sh_assets_staff_costs,wgt
0,4Children,R,1.0,2011,2880902.0,9603182.0,91404.0,310947.0,12886435.0,12127472.0,...,NaN,NaN,0.072636,0.135971,0.767809,0.094602,0.177090,0.534203,5.646843,1.0
1,4Children,R,1.0,2014,6810520.0,18768904.0,58002.0,401879.0,26039304.0,25493796.0,...,NaN,NaN,0.057641,0.089150,1.001396,0.057560,0.089026,0.646561,11.232729,1.0
2,4Children,R,1.0,2015,7199403.0,21638036.0,132191.0,512654.0,29482284.0,32290108.0,...,NaN,NaN,-0.049619,-0.079828,-0.620210,0.080004,0.128711,0.621583,7.769365,1.0
3,4Children,R,1.0,2013,5573013.0,15194731.0,228844.0,267156.0,21263744.0,20989048.0,...,NaN,NaN,0.045740,0.068251,1.008259,0.045365,0.067692,0.670166,14.772749,1.0
4,4Children,R,1.0,2010,2056816.0,7335103.0,110256.0,424628.0,9926803.0,9769816.0,...,NaN,NaN,0.057696,0.122532,0.567539,0.101660,0.215901,0.470862,4.631749,1.0


### Pandas crosstab

In [6]:
table = pd.crosstab(df.year, df.grant_type)
table

grant_type,G,N,R,R/G
year,,,,
2010,15,59,71,8
2011,15,59,71,8
2012,15,59,71,8
2013,15,59,71,8
2014,15,59,71,8
2015,15,59,71,8


### ACRO crosstab

In [7]:
safe_table = acro.crosstab(df.year, df.grant_type)
safe_table

INFO:acro:get_summary(): fail; threshold: 6 cells may need suppressing; 
INFO:acro:outcome_df:
grant_type   G   N   R          R/G
year                               
2010        ok  ok  ok  threshold; 
2011        ok  ok  ok  threshold; 
2012        ok  ok  ok  threshold; 
2013        ok  ok  ok  threshold; 
2014        ok  ok  ok  threshold; 
2015        ok  ok  ok  threshold; 
INFO:acro:records:add(): output_0


grant_type,G,N,R,R/G
year,,,,
2010,15,59,71,8
2011,15,59,71,8
2012,15,59,71,8
2013,15,59,71,8
2014,15,59,71,8
2015,15,59,71,8


### ACRO crosstab with aggregation function

In [8]:
safe_table = acro.crosstab(df.year, df.grant_type, values=df.inc_grants, aggfunc="mean")
safe_table

INFO:acro:get_summary(): fail; threshold: 6 cells may need suppressing; p-ratio: 1 cells may need suppressing; nk-rule: 1 cells may need suppressing; 
INFO:acro:outcome_df:
grant_type   G   N   R                            R/G
year                                                 
2010        ok  ok  ok  threshold; p-ratio; nk-rule; 
2011        ok  ok  ok                    threshold; 
2012        ok  ok  ok                    threshold; 
2013        ok  ok  ok                    threshold; 
2014        ok  ok  ok                    threshold; 
2015        ok  ok  ok                    threshold; 
INFO:acro:records:add(): output_1


grant_type,G,N,R,R/G
year,,,,
2010,9921906.0,0.000000,8402284.0,11636000.0
2011,8502247.0,124013.859375,7716880.0,16047500.0
2012,11458580.0,131859.062500,6958050.5,16810000.0
2013,13557147.0,147937.796875,7202273.5,16765625.0
2014,13748147.0,133198.250000,8277525.5,17845750.0
2015,11133433.0,146572.187500,10812888.0,18278624.0


### ACRO crosstab with multiple aggregation functions and totals

In [9]:
safe_table = acro.crosstab(
    df.year, df.grant_type, values=df.inc_grants, aggfunc=["mean", "std"], margins=True
)
safe_table

INFO:acro:get_summary(): fail; threshold: 12 cells may need suppressing; p-ratio: 2 cells may need suppressing; nk-rule: 2 cells may need suppressing; 
INFO:acro:outcome_df:
           mean                                            std          \
grant_type    G   N   R                            R/G All   G   N   R   
year                                                                     
2010         ok  ok  ok  threshold; p-ratio; nk-rule;   ok  ok  ok  ok   
2011         ok  ok  ok                    threshold;   ok  ok  ok  ok   
2012         ok  ok  ok                    threshold;   ok  ok  ok  ok   
2013         ok  ok  ok                    threshold;   ok  ok  ok  ok   
2014         ok  ok  ok                    threshold;   ok  ok  ok  ok   
2015         ok  ok  ok                    threshold;   ok  ok  ok  ok   
All          ok  ok  ok                             ok  ok  ok  ok  ok   

                                               
grant_type                           

mean                                                    \
grant_type           G              N           R         R/G        All   
year                                                                       
2010         9921906.0       0.000000   8402284.0  11636000.0  8308286.0   
2011         8502247.0  124013.859375   7716880.0  16047500.0  5303808.0   
2012        11458580.0  131859.062500   6958050.5  16810000.0  5259893.0   
2013        13557147.0  147937.796875   7202273.5  16765625.0  5605045.5   
2014        13748147.0  133198.250000   8277525.5  17845750.0  6117054.5   
2015        11133433.0  146572.187500  10812888.0  18278624.0  6509989.5   
All         11412787.0  134431.890625   8098502.0  16648273.0  5997796.5   

                     std                                             \
grant_type             G              N             R           R/G   
year                                                                  
2010        1.855055e+07       0.000000  3.059557e+07  1.701088e+07   
2011        1.688595e+07  205959.492903  2.954322e+07  1.561638e+07   
2012        2.061090e+07  210476.539175  2.721184e+07  1.646449e+07   
2013        2.486844e+07  203747.417017  2.989833e+07  1.671112e+07   
2014        3.134559e+07  181865.925580  3.546348e+07  1.741251e+07   
2015        2.553919e+07  201602.800832  4.130935e+07  1.730471e+07   
All         2.283220e+07  198873.726656  3.204495e+07  1.583532e+07   

                          
grant_type           All  
year                      
2010        2.727398e+07  
2011        2.137658e+07  
2012        2.026400e+07  
2013        2.251787e+07  
2014        2.641722e+07  
2015        2.784636e+07  
All         2.403848e+07

### ACRO crosstab with missing values

In [10]:
utils.CHECK_MISSING_VALUES = True

missing = df.inc_grants.copy()
missing[0:10] = np.NaN

safe_table = acro.crosstab(df.year, df.grant_type, values=missing, aggfunc="mean")
safe_table

INFO:acro:get_summary(): review; missing values found
INFO:acro:outcome_df:
grant_type        G        N        R      R/G
year                                          
2010        missing  missing  missing  missing
2011                 missing  missing         
2012                          missing         
2013                 missing  missing         
2014                 missing  missing         
2015        missing  missing  missing         
INFO:acro:records:add(): output_3


grant_type,G,N,R,R/G
year,,,,
2010,9921906.0,0.000000,8420372.0,11636000.0
2011,8502247.0,125663.226562,7689140.5,16047500.0
2012,11458580.0,131859.062500,6896304.0,16810000.0
2013,13557147.0,150488.453125,7088096.0,16765625.0
2014,13748147.0,135494.781250,8118565.0,17845750.0
2015,11133433.0,149143.625000,10596385.0,18278624.0


In [11]:
utils.CHECK_MISSING_VALUES = False

### ACRO crosstab with negative values

In [12]:
negative = df.inc_grants.copy()
negative[0:10] = -10

safe_table = acro.crosstab(df.year, df.grant_type, values=negative, aggfunc="mean")
safe_table

INFO:acro:get_summary(): review; negative values found
INFO:acro:outcome_df:
grant_type G         N         R R/G
year                                
2010                    negative    
2011          negative  negative    
2012                    negative    
2013          negative  negative    
2014          negative  negative    
2015          negative  negative    
INFO:acro:records:add(): output_4


grant_type,G,N,R,R/G
year,,,,
2010,9921906.0,0.000000,8280032.5,11636000.0
2011,8502247.0,123496.445312,7577703.5,16047500.0
2012,11458580.0,131859.062500,6796357.5,16810000.0
2013,13557147.0,147937.625000,6988263.5,16765625.0
2014,13748147.0,133198.078125,7997392.5,17845750.0
2015,11133433.0,146572.015625,10388613.0,18278624.0


### ACRO pivot_table

In [13]:
table = acro.pivot_table(
    df, index=["grant_type"], values=["inc_grants"], aggfunc=["mean", "std"]
)
table

INFO:acro:get_summary(): pass
INFO:acro:outcome_df:
                 mean        std
           inc_grants inc_grants
grant_type                      
G                  ok         ok
N                  ok         ok
R                  ok         ok
R/G                ok         ok
INFO:acro:records:add(): output_5


,mean,std
,inc_grants,inc_grants
grant_type,,
G,1.141279e+07,2.283220e+07
N,1.344319e+05,1.988737e+05
R,8.098502e+06,3.204495e+07
R/G,1.664827e+07,1.583532e+07


### ACRO pivot_table with missing values

In [14]:
utils.CHECK_MISSING_VALUES = True

df.loc[0:10, "inc_grants"] = np.NaN

table = acro.pivot_table(
    df, index=["grant_type"], values=["inc_grants"], aggfunc=["mean", "std"]
)
table

INFO:acro:get_summary(): review; missing values found
INFO:acro:outcome_df:
                 mean        std
           inc_grants inc_grants
grant_type                      
G             missing    missing
N             missing    missing
R             missing    missing
R/G           missing    missing
INFO:acro:records:add(): output_6


,mean,std
,inc_grants,inc_grants
grant_type,,
G,1.141279e+07,2.283220e+07
N,1.364700e+05,1.999335e+05
R,8.006360e+06,3.228216e+07
R/G,1.664827e+07,1.583532e+07


In [15]:
utils.CHECK_MISSING_VALUES = False

### ACRO pivot_table with negative values

In [16]:
df.loc[0:10, "inc_grants"] = -10

table = acro.pivot_table(
    df, index=["grant_type"], values=["inc_grants"], aggfunc=["mean", "std"]
)
table

INFO:acro:get_summary(): review; negative values found
INFO:acro:outcome_df:
                 mean        std
           inc_grants inc_grants
grant_type                      
G                               
N            negative   negative
R            negative   negative
R/G                             
INFO:acro:records:add(): output_7


,mean,std
,inc_grants,inc_grants
grant_type,,
G,1.141279e+07,2.283220e+07
N,1.341800e+05,1.990196e+05
R,7.882231e+06,3.204558e+07
R/G,1.664827e+07,1.583532e+07


### ACRO OLS

In [17]:
new_df = df[["inc_activity", "inc_grants", "inc_donations", "total_costs"]]
new_df = new_df.dropna()

y = new_df["inc_activity"]
x = new_df[["inc_grants", "inc_donations", "total_costs"]]
x = add_constant(x)

results = acro.ols(y, x)
results.summary()

INFO:acro:ols() outcome: pass; dof=807.0 >= 10


INFO:acro:records:add(): output_8


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           inc_activity   R-squared:                       0.894
Model:                            OLS   Adj. R-squared:                  0.893
Method:                 Least Squares   F-statistic:                     2261.
Date:                Wed, 26 Jul 2023   Prob (F-statistic):               0.00
Time:                        14:49:26   Log-Likelihood:                -14495.
No. Observations:                 811   AIC:                         2.900e+04
Df Residuals:                     807   BIC:                         2.902e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const           3.01e+05   5.33e+05      0.565      0.572   -7.45e+05    1.35e+06
inc_grants       -0.8846      0.025    -35.956      0.000      -0.933      -0.836
inc_donations    -0.6647      0.016    -40.721      0.000      -0.697      -0.633
total_costs       0.8313      0.011     78.674      0.000       0.811       0.852
==============================================================================
Omnibus:                     1339.956   Durbin-Watson:                   1.414
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1253317.706
Skew:                           9.899   Prob(JB):                         0.00
Kurtosis:                     194.566   Cond. No.                     1.05e+08
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.05e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### ACRO OLSR

In [18]:
results = acro.olsr(
    formula="inc_activity ~ inc_grants + inc_donations + total_costs", data=new_df
)
results.summary()

INFO:acro:olsr() outcome: pass; dof=807.0 >= 10


INFO:acro:records:add(): output_9


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           inc_activity   R-squared:                       0.894
Model:                            OLS   Adj. R-squared:                  0.893
Method:                 Least Squares   F-statistic:                     2261.
Date:                Wed, 26 Jul 2023   Prob (F-statistic):               0.00
Time:                        14:49:26   Log-Likelihood:                -14495.
No. Observations:                 811   AIC:                         2.900e+04
Df Residuals:                     807   BIC:                         2.902e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept       3.01e+05   5.33e+05      0.565      0.572   -7.45e+05    1.35e+06
inc_grants       -0.8846      0.025    -35.956      0.000      -0.933      -0.836
inc_donations    -0.6647      0.016    -40.721      0.000      -0.697      -0.633
total_costs       0.8313      0.011     78.674      0.000       0.811       0.852
==============================================================================
Omnibus:                     1339.956   Durbin-Watson:                   1.414
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1253317.706
Skew:                           9.899   Prob(JB):                         0.00
Kurtosis:                     194.566   Cond. No.                     1.05e+08
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.05e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### ACRO Probit

In [19]:
new_df = df[["survivor", "inc_activity", "inc_grants", "inc_donations", "total_costs"]]
new_df = new_df.dropna()

y = new_df["survivor"].astype("category").cat.codes  # numeric
y.name = "survivor"
x = new_df[["inc_activity", "inc_grants", "inc_donations", "total_costs"]]
x = add_constant(x)

results = acro.probit(y, x)
results.summary()

INFO:acro:probit() outcome: pass; dof=806.0 >= 10


INFO:acro:records:add(): output_10


Optimization terminated successfully.
         Current function value: 0.493791
         Iterations 10


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Probit Regression Results                           
==============================================================================
Dep. Variable:               survivor   No. Observations:                  811
Model:                         Probit   Df Residuals:                      806
Method:                           MLE   Df Model:                            4
Date:                Wed, 26 Jul 2023   Pseudo R-squ.:                  0.2140
Time:                        14:49:27   Log-Likelihood:                -400.46
converged:                       True   LL-Null:                       -509.50
Covariance Type:            nonrobust   LLR p-value:                 4.875e-46
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.0474      0.057      0.838      0.402      -0.063       0.158
inc_activity   1.836e-07   5.16e-08      3.559      0.000    8.25e-08    2.85e-07
inc_grants     8.576e-08    3.9e-08      2.197      0.028    9.25e-09    1.62e-07
inc_donations  2.406e-07   4.54e-08      5.297      0.000    1.52e-07     3.3e-07
total_costs   -8.644e-08   3.68e-08     -2.351      0.019   -1.59e-07   -1.44e-08
=================================================================================

Possibly complete quasi-separation: A fraction 0.18 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

### ACRO Logit

In [20]:
results = acro.logit(y, x)
results.summary()

INFO:acro:logit() outcome: pass; dof=806.0 >= 10


INFO:acro:records:add(): output_11


Optimization terminated successfully.
         Current function value: 0.490836
         Iterations 12


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               survivor   No. Observations:                  811
Model:                          Logit   Df Residuals:                      806
Method:                           MLE   Df Model:                            4
Date:                Wed, 26 Jul 2023   Pseudo R-squ.:                  0.2187
Time:                        14:49:27   Log-Likelihood:                -398.07
converged:                       True   LL-Null:                       -509.50
Covariance Type:            nonrobust   LLR p-value:                 4.532e-47
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.0512      0.091      0.561      0.575      -0.128       0.230
inc_activity   2.981e-07   8.95e-08      3.330      0.001    1.23e-07    4.74e-07
inc_grants     1.351e-07   6.67e-08      2.026      0.043    4.39e-09    2.66e-07
inc_donations  5.123e-07   1.04e-07      4.927      0.000    3.08e-07    7.16e-07
total_costs   -1.442e-07   6.26e-08     -2.304      0.021   -2.67e-07   -2.15e-08
=================================================================================

Possibly complete quasi-separation: A fraction 0.18 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

### List current ACRO outputs

In [21]:
results_str = acro.print_outputs()

uid: output_0
status: fail
type: table
properties: {'method': 'crosstab'}
sdc: {'summary': {'suppressed': False, 'negative': 0, 'missing': 0, 'threshold': 6, 'p-ratio': 0, 'nk-rule': 0}, 'cells': {'negative': [], 'missing': [], 'threshold': [[0, 3], [1, 3], [2, 3], [3, 3], [4, 3], [5, 3]], 'p-ratio': [], 'nk-rule': []}}
command: safe_table = acro.crosstab(df.year, df.grant_type)
summary: fail; threshold: 6 cells may need suppressing; 
outcome: grant_type   G   N   R          R/G
year                               
2010        ok  ok  ok  threshold; 
2011        ok  ok  ok  threshold; 
2012        ok  ok  ok  threshold; 
2013        ok  ok  ok  threshold; 
2014        ok  ok  ok  threshold; 
2015        ok  ok  ok  threshold; 
output: [grant_type   G   N   R  R/G
year                       
2010        15  59  71    8
2011        15  59  71    8
2012        15  59  71    8
2013        15  59  71    8
2014        15  59  71    8
2015        15  59  71    8]
timestamp: 2023-07-26T14:49:25

### Remove some ACRO outputs before finalising

In [22]:
acro.remove_output("output_1")
acro.remove_output("output_4")

INFO:acro:records:remove(): output_1 removed
INFO:acro:records:remove(): output_4 removed


### Rename ACRO outputs before finalising

In [23]:
acro.rename_output("output_2", "pivot_table")

INFO:acro:records:rename_output(): output_2 renamed to pivot_table


### Add a comment to output

In [24]:
acro.add_comments("output_0", "This is a cross table between year and grant_type")
acro.add_comments("output_0", "6 cells were suppressed in this table")

INFO:acro:records:a comment was added to output_0
INFO:acro:records:a comment was added to output_0


### Add an unsupported output to the list of outputs

In [25]:
acro.custom_output(
    "XandY.jpeg", "This output is an image showing the relationship between X and Y"
)

INFO:acro:records:add_custom(): output_12


### Request an exception for some of the outputs

In [26]:
acro.add_exception("output_0", "I really need this.")
acro.add_exception("output_3", "This one is safe. Trust me, I'm a professor.")
acro.add_exception("output_5", "It's not disclosive, I promise.")
acro.add_exception("output_6", "I need this one too")

INFO:acro:records:exception request was added to output_0
INFO:acro:records:exception request was added to output_3
INFO:acro:records:exception request was added to output_5
INFO:acro:records:exception request was added to output_6


### Finalise ACRO

In [27]:
SAVE_PATH = "ACRO_RES"

# output = acro.finalise(SAVE_PATH, "xlsx")
output = acro.finalise(SAVE_PATH, "json")

INFO:acro:records:
uid: output_7
status: review
type: table
properties: {'method': 'pivot_table'}
sdc: {'summary': {'suppressed': False, 'negative': 4, 'missing': 0, 'threshold': 0, 'p-ratio': 0, 'nk-rule': 0}, 'cells': {'negative': [[1, 0], [1, 1], [2, 0], [2, 1]], 'missing': [], 'threshold': [], 'p-ratio': [], 'nk-rule': []}}
command: table = acro.pivot_table(
summary: review; negative values found
outcome:                  mean        std
           inc_grants inc_grants
grant_type                      
G                               
N            negative   negative
R            negative   negative
R/G                             
output: [                    mean           std
              inc_grants    inc_grants
grant_type                            
G           1.141279e+07  2.283220e+07
N           1.341800e+05  1.990196e+05
R           7.882231e+06  3.204558e+07
R/G         1.664827e+07  1.583532e+07]
timestamp: 2023-07-26T14:49:26.729368
comments: []
exception: 

The statu

### List files generated

In [28]:
files = []
for name in os.listdir(SAVE_PATH):
    if os.path.isfile(os.path.join(SAVE_PATH, name)):
        files.append(name)
files.sort()
for f in files:
    print(f)

XandY.jpeg
config.json
output_0_0.csv
output_10_0.csv
output_10_1.csv
output_11_0.csv
output_11_1.csv
output_3_0.csv
output_5_0.csv
output_6_0.csv
output_7_0.csv
output_8_0.csv
output_8_1.csv
output_8_2.csv
output_9_0.csv
output_9_1.csv
output_9_2.csv
pivot_table_0.csv
results.json


### Checksums

In [29]:
files = []
checksum_dir = os.path.join(SAVE_PATH, "checksums")
for name in os.listdir(checksum_dir):
    if os.path.isfile(os.path.join(checksum_dir, name)):
        files.append(name)
files.sort()
for f in files:
    print(f)

XandY.jpeg.txt
output_0_0.csv.txt
output_10_0.csv.txt
output_10_1.csv.txt
output_11_0.csv.txt
output_11_1.csv.txt
output_3_0.csv.txt
output_5_0.csv.txt
output_6_0.csv.txt
output_7_0.csv.txt
output_8_0.csv.txt
output_8_1.csv.txt
output_8_2.csv.txt
output_9_0.csv.txt
output_9_1.csv.txt
output_9_2.csv.txt
pivot_table_0.csv.txt
results.json.txt


### Inspect metadata

In [30]:
results_json = os.path.normpath(f"{SAVE_PATH}/results.json")
with open(results_json, "r") as f:
    print(f.read())

{
    "version": "0.4.2",
    "results": {
        "output_0": {
            "uid": "output_0",
            "status": "fail",
            "type": "table",
            "properties": {
                "method": "crosstab"
            },
            "files": [
                {
                    "name": "output_0_0.csv",
                    "sdc": {
                        "summary": {
                            "suppressed": false,
                            "negative": 0,
                            "missing": 0,
                            "threshold": 6,
                            "p-ratio": 0,
                            "nk-rule": 0
                        },
                        "cells": {
                            "negative": [],
                            "missing": [],
                            "threshold": [
                                [
                                    0,
                                    3
                                ],
             